# Building AI Applications with `Gradio`

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**In this repository, you will find examples of to create simple interfaces for things like, for example, [sentiment analysis](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/bbe9c0a77499fa68de7c6d53bf5ef7e0b43a25e0/ML%20Explainability/NLP%20Interpreter%20(en)/senti_dash_en.py), and even a [playground for HuggingFace language models](https://github.com/Nkluge-correa/teeny-tiny_castle/tree/master/ML%20Explainability/NLP%20Playgroung). These interfaces can be used to create ML applications since all of them use things like `Flask` for the backend, and bootstrap and CSS for the front end part.**

**However, there are simpler ways to create demo ML apps. And one of the simpler ways is `Gradio`.**

**[Gradio](https://gradio.app/) is a free and open-source Python library that allows you to develop an easy-to-use customizable component demo for your machine learning model that anyone can use anywhere. Gradio integrates with the most popular Python libraries used for ML and Data Science, including [Scikit-learn](https://scikit-learn.org/stable/), [PyTorch](https://pytorch.org/), [NumPy](https://numpy.org/), [seaborn](https://seaborn.pydata.org/), [pandas](https://pandas.pydata.org/), [TensorFlow](https://www.tensorflow.org/), and many others.**

![gradio-image](https://pbs.twimg.com/profile_images/1526964416834510848/Njy4Kh2q_400x400.jpg)

**Let's first create an application to [recognize digits](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/bbe9c0a77499fa68de7c6d53bf5ef7e0b43a25e0/ML%20Intro%20Course/MNIST_digit.ipynb), one of the first tasks in DL that we explore in our mini course.**

**For this application, instead of training a dense feed-forward model, we will train a convolutional neural network (CNN), since this is the standard in computer vision applications. You can find other examples of how to build CNNs on [this](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/bbe9c0a77499fa68de7c6d53bf5ef7e0b43a25e0/ML%20Explainability/CV%20Interpreter/CNN_model_maker.ipynb) notebook.**

In [1]:
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_images=x_train.reshape(x_train.shape[0], 28, 28, 1)
test_images=x_test.reshape(x_test.shape[0], 28, 28 ,1) 
                                            
train_labels=tf.keras.utils.to_categorical(y_train)
test_labels=tf.keras.utils.to_categorical(y_test)

print('Training Set Size: '), print(x_train.shape)
print('Test Set Size: '), print(x_test.shape)

model = keras.models.Sequential([
    keras.layers.Conv2D(20, (5,5), padding='same', activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Conv2D(50, (5,5), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
model.summary()

print('Training...\n')
history = model.fit(train_images, train_labels, epochs=10,
                    batch_size=256, verbose=2)
print('\nEvaluating...\n')
test_loss_score, test_acc_score = model.evaluate(test_images, test_labels)
print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')
model.save('models\digit_classifier.h5')

11490434/11490434 [==============================] - 1s 0us/step
Training Set Size: 
(60000, 28, 28)
Test Set Size: 
(10000, 28, 28)
Version:  2.10.0
Eager mode:  True
GPU is available
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        25050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 50)         0         
 2D)                                                             
                                                                 
 fl

**With our trained and saved model, creating an application with gradio takes no more than a little more than 20 lines of code. You can style certain components with HTML, Markdown, and CSS. Applications created with gradio can even be hosted on the `gradio.app` for 72 hours, free of charge. Just use launch the application with the `share` argument equal to `True` (`demo.launch(share=True)`).**

**Permanent hosting can be easily done through the [HuggingFace Spaces](https://www.huggingface.co/spaces), or any PaaS (Platforms as a service) you wish to use, like [Heroku](https://heroku.com/) or [Render](https://render.com/).**

In [ ]:
model = keras.models.load_model('models\digit_classifier.h5')
classes = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

def predict(img):
	prediction = model.predict(img.reshape(1, 28, 28, 1), verbose=0).tolist()[0]
	return {classes[i]: prediction[i] for i in range(10)}

import gradio as gr

title = "Digit Classifier - By Teeny-Tiny Castle 🏰"

head = (
  "<center>"
  "<img src='https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png' width=400>"
  "This model was trained to classify numbers (from 0 to 9). To test it, write your number in the space provided."
  "</center>"
)

ref = "Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle)."


demo = gr.Interface(fn=predict, 
             inputs="sketchpad",
             outputs=gr.Label(num_top_classes=3),
             allow_flagging="never",
             title=title, 
             description=head, 
             article=ref)
if __name__ == "__main__":
    demo.launch(share=True)

**You can even use Gradio to create applications that are not ML-based. For example, below we show you how to create a closed-domain chatbot with a small number of answers and questions (_a basic rules-based system that performs n-gram search_). To see the full implementation of this bot, go to [this](https://github.com/Nkluge-correa/Aira-EXPERT) link. and to chat with `Ai.ra` go to [this](https://aira-expert.onrender.com/) link!**

In [ ]:
from statistics import mode
import urllib.request
import gradio as gr
import itertools
import unidecode
import string
import json



urllib.request.urlretrieve(
    'https://drive.google.com/uc?export=download&id=1TXD41vfqNWA6UNDQ73WhtJdZivCJatn-', 
    'bot__questions_tags.json'
)

answers = ['Hello! My name is Ai.ra, and I am an artificial intelligence (AI). More specifically, I am an NLP (Natural Language Processing) model trained in conversation (a chatbot!).',
 'You can ask me things about "Artificial Intelligence," "Machine Learning," "AI Safety," or "AI Ethics."',
 "I don't have that kind of property hahaha I am software!",
 "AIRES (AI Robotics Ethics Society) is a society focused on educating the leaders and developers of tomorrow's Artificial Intelligence (AI) to ensure that AI is created ethically and responsibly.",
 'Aron Hui is the president/founder of AIRES.',
 'What "intelligence" is, remains an open question. However, not to leave you in the lurch, I will define "intelligence" as follows: "Intelligence is the ability of an agent to achieve goals in a wide range of environments."',
 'There is no consensus in the literature on what "AI" is (a corollary of not having a robust definition of what "intelligence\'\' is). However, we can say that AI is the intelligence demonstrated by machines, as opposed to the natural intelligence possessed by animals and humans.',
 'General Intelligence, or Universal Intelligence, can be defined as the ability to efficiently achieve goals in a wide range of domains.',
 'GOFAI ("good-old-fashioned-ai"), or symbolic artificial intelligence, is the term used to refer to methods of developing AI systems based on high-level symbolic (interpretable) representations, logic, and search.',
 'A multi-agent system (MAS "Multi-Agent Systems") is a computer system composed of multiple interacting intelligent agents.',
 'Machine Learning (ML) is a field of research dedicated to understanding and building methods that "learn", i.e., methods that use information/data to improve performance on some tasks.']


with open('bot__questions_tags.json') as json_file:
    dictionary = json.load(json_file)


def generate_ngrams(text, WordsToCombine):
    words = text.split()
    output = []
    for i in range(len(words) - WordsToCombine+1):
        output.append(words[i:i+WordsToCombine])
    return output


def make_keys(text, WordsToCombine):
    gram = generate_ngrams(text, WordsToCombine)
    sentences = []
    for i in range(0, len(gram)):
        sentence = ' '.join(gram[i])
        sentences.append(sentence)
    return sentences


def chat(message, history):
    history = history or []
    message = message.lower()
    sentences = []
    values = []
    new_text = message.translate(str.maketrans('', '', string.punctuation))
    new_text = unidecode.unidecode(new_text)

    if len(new_text.split()) == 1:
        if new_text in dictionary.keys():
            l = [dictionary[new_text]] * 100
            values.append(l)
        new_text = new_text + ' ' + new_text

    elif len(new_text.split()) != 1:
        if new_text in dictionary.keys():
            l = [dictionary[new_text]] * 100
            values.append(l)
    else:
        pass

    for i in range(1, len(new_text.split()) + 1):
        sentence = make_keys(new_text, i)
        sentences.append(sentence)

    for i in range(0, len(sentences)):
        attention = sentences[i]
        for i in range(0, len(attention)):
            if attention[i] in dictionary.keys():
                l = [dictionary[attention[i]]] * i
                values.append(l)

    if len(values) == 0:
        bot_input_ids = "I'm sorry, either I didn't understand the question, or it is not part of my domain of expertise... :( Try asking it in another way or using other words. Maybe then I can help you!"
        history.append((message, bot_input_ids))
        return history, history

    elif len(values) != 0:
        values = list(itertools.chain(*values))
        prediction = mode(values)
        bot_input_ids = answers[int(prediction)-1]
        history.append((message, bot_input_ids))
        return history, history

title = "Basic Chatbot - By Teeny-Tiny Castle 🏰"

head = (
  "<center>"
  "<img src='https://d2vrvpw63099lz.cloudfront.net/do-i-need-a-chatbot/header-chat-box.png' width=400>"
  "This is an example of a rulses-based close domain chatbot. It knows a couple of awensers to questions related to AI."
  "<br>"
  "</center>"
)

ref = " To see its full version (ML style) of this bot, go to [this](https://aira-expert.onrender.com/) link."

chatbot = gr.Chatbot().style(color_map=("green", "pink"))
demo = gr.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
    title=title, 
    description=head, 
    article=ref
)
if __name__ == "__main__":
    demo.launch(share=True)

**Now you know how to create simple AI applications to show and share with your friends and colleagues!** 🤖

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).